In [1]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [2]:
%pip install docx2txt
import docx2txt
Text = docx2txt.process("/content/drive/MyDrive/B.TechProject/test data.docx")

  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3960 sha256=647e67b4af60868ac7b69bed883f2f0cd5250f11868d954099d92651bf2023bb
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt


In [3]:
TextOne=Text.split("Content",1)[1]
TextT=TextOne.split("Course Outcome:",1)[0]
Text=TextT

In [5]:
import nltk
from nltk import word_tokenize
import string

nltk.download('punkt')
# add below lines
nltk.download('punkt_tab')

def clean(text):
    text = text.lower()
    printable = set(string.printable)
    text = filter(lambda x: x in printable, text)
    text = "".join(list(text))
    return text

Cleaned_text = clean(Text)
text = word_tokenize(Cleaned_text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Groping with POS TAG
List of POS_TAGS in NLTK


In [8]:
nltk.download('averaged_perceptron_tagger_eng')

POS_tag = nltk.pos_tag(text)



[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


In [9]:
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer

wordnet_lemmatizer = WordNetLemmatizer()

adjective_tags = ['JJ','JJR','JJS']

lemmatized_text = []

for word in POS_tag:
    if word[1] in adjective_tags:
        lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0],pos="a")))
    else:
        lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0]))) #default POS = noun



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [10]:
lemmatized_text=lemmatized_text[2:]
POS_tag = nltk.pos_tag(lemmatized_text)

Get Rid Off from STOP WORDS

In [11]:
stopwords = []

wanted_POS = ['NN','NNS','NNP','NNPS','JJ','JJR','JJS','VBG','FW']

for word in POS_tag:
    if word[1] not in wanted_POS:
        stopwords.append(word[0])

punctuations = list(str(string.punctuation))

stopwords = stopwords + punctuations

In [13]:
stopword_file = open("/content/drive/MyDrive/B.TechProject/stop_words.txt", "r")

lots_of_stopwords = []

for line in stopword_file.readlines():
    lots_of_stopwords.append(str(line.strip()))

stopwords_plus = []
stopwords_plus = stopwords + lots_of_stopwords
stopwords_plus = set(stopwords_plus)

#Stopwords_plus contain total set of all stopwords

In [14]:
processed_text = []
for word in lemmatized_text:
    if word not in stopwords_plus:
        processed_text.append(word)


In [15]:
vocabulary = list(set(processed_text))


In [16]:
import numpy as np
import math
vocab_len = len(vocabulary)

weighted_edge = np.zeros((vocab_len,vocab_len),dtype=np.float32)

score = np.zeros((vocab_len),dtype=np.float32)
window_size = 3
covered_coocurrences = []

for i in range(0,vocab_len):
    score[i]=1
    for j in range(0,vocab_len):
        if j==i:
            weighted_edge[i][j]=0
        else:
            for window_start in range(0,(len(processed_text)-window_size)):

                window_end = window_start+window_size

                window = processed_text[window_start:window_end]

                if (vocabulary[i] in window) and (vocabulary[j] in window):

                    index_of_i = window_start + window.index(vocabulary[i])
                    index_of_j = window_start + window.index(vocabulary[j])

                    # index_of_x is the absolute position of the xth term in the window
                    # (counting from 0)
                    # in the processed_text

                    if [index_of_i,index_of_j] not in covered_coocurrences:
                        weighted_edge[i][j]+=1/math.fabs(index_of_i-index_of_j)
                        covered_coocurrences.append([index_of_i,index_of_j])



In [17]:
inout = np.zeros((vocab_len),dtype=np.float32)

for i in range(0,vocab_len):
    for j in range(0,vocab_len):
        inout[i]+=weighted_edge[i][j]

In [18]:
MAX_ITERATIONS = 50
d=0.85
threshold = 0.0001 #convergence threshold

for iter in range(0,MAX_ITERATIONS):
    prev_score = np.copy(score)

    for i in range(0,vocab_len):

        summation = 0
        for j in range(0,vocab_len):
            if weighted_edge[i][j] != 0:
                summation += (weighted_edge[i][j]/inout[j])*score[j]

        score[i] = (1-d) + d*(summation)

    if np.sum(np.fabs(prev_score-score)) <= threshold: #convergence condition
        print("Converging at iteration "+str(iter)+"....")
        break

Converging at iteration 28....


In [19]:
for i in range(0,vocab_len):
    print("Score of "+vocabulary[i]+": "+str(score[i]))

Score of management: 2.8473296
Score of process: 0.69745016
Score of attribute: 0.65941054
Score of tree: 0.8268794
Score of dfds: 0.92696106
Score of scheduling: 0.66623694
Score of critical: 0.84934855
Score of acceptance: 0.74419236
Score of white-box: 0.7240703
Score of logical: 0.9212333
Score of er: 0.86946756
Score of concept: 0.6962515
Score of repair: 0.78555673
Score of mttf: 0.80357563
Score of software: 7.5382953
Score of wb: 0.7820784
Score of level: 0.6936784
Score of reliability: 1.4522562
Score of total: 0.6629527
Score of regression: 0.7123163
Score of feasibility: 0.73151153
Score of project: 2.092928
Score of standard: 0.6719456
Score of estimation-cocomo: 0.77136934
Score of black-box: 0.7319427
Score of objective: 0.73616683
Score of cyclomatic: 0.7552928
Score of spiral: 0.7044086
Score of context: 0.78955626
Score of data: 0.92124915
Score of agile: 0.6897708
Score of engineering: 0.74720126
Score of test: 0.76499444
Score of cost: 0.73701704
Score of sr: 0.73774

In [20]:
phrases = []

phrase = " "
for word in lemmatized_text:

    if word in stopwords_plus:
        if phrase!= " ":
            phrases.append(str(phrase).strip().split())
        phrase = " "
    elif word not in stopwords_plus:
        phrase+=str(word)
        phrase+=" "

print("Partitioned Phrases (Candidate Keyphrases): \n")
print(phrases)

Partitioned Phrases (Candidate Keyphrases): 

[['mark'], ['information', 'system'], ['software', 'engineering', 'objective'], ['definition'], ['software', 'development', 'life', 'cycle'], ['software', 'process', 'model', 'waterfall', 'model'], ['spiral', 'model'], ['agile', 'model'], ['software', 'requirement'], ['sr'], ['feasibility', 'analysis'], ['software', 'design'], ['context', 'diagram'], ['dfd'], ['physical'], ['logical', 'dfds'], ['data', 'dictionary'], ['er', 'diagram'], ['decision', 'tree'], ['decision', 'table'], ['structure', 'chart'], ['english'], ['software', 'testing'], ['level'], ['testing'], ['white-box'], ['black-box', 'testing'], ['test', 'case', 'generation'], ['acceptance', 'testing'], ['software', 'validation'], ['regression', 'testing'], ['mutation', 'analysis'], ['cyclomatic', 'complexity'], ['reliability'], ['reliability', 'concept'], ['software', 'reliability'], ['hazard'], ['mttf'], ['mtbf'], ['repair'], ['availability'], ['software', 'quality'], ['quality',

In [21]:
unique_phrases = []

for phrase in phrases:
    if phrase not in unique_phrases:
        unique_phrases.append(phrase)

print("Unique Phrases (Candidate Keyphrases): \n")
print(unique_phrases)

Unique Phrases (Candidate Keyphrases): 

[['mark'], ['information', 'system'], ['software', 'engineering', 'objective'], ['definition'], ['software', 'development', 'life', 'cycle'], ['software', 'process', 'model', 'waterfall', 'model'], ['spiral', 'model'], ['agile', 'model'], ['software', 'requirement'], ['sr'], ['feasibility', 'analysis'], ['software', 'design'], ['context', 'diagram'], ['dfd'], ['physical'], ['logical', 'dfds'], ['data', 'dictionary'], ['er', 'diagram'], ['decision', 'tree'], ['decision', 'table'], ['structure', 'chart'], ['english'], ['software', 'testing'], ['level'], ['testing'], ['white-box'], ['black-box', 'testing'], ['test', 'case', 'generation'], ['acceptance', 'testing'], ['software', 'validation'], ['regression', 'testing'], ['mutation', 'analysis'], ['cyclomatic', 'complexity'], ['reliability'], ['reliability', 'concept'], ['software', 'reliability'], ['hazard'], ['mttf'], ['mtbf'], ['repair'], ['availability'], ['software', 'quality'], ['quality', 'att

In [22]:
for word in vocabulary:
    #print word
    for phrase in unique_phrases:
        if (word in phrase) and ([word] in unique_phrases) and (len(phrase)>1):
            #if len(phrase)>1 then the current phrase is multi-worded.
            #if the word in vocabulary is present in unique_phrases as a single-word-phrase
            # and at the same time present as a word within a multi-worded phrase,
            # then I will remove the single-word-phrase from the list.
            unique_phrases.remove([word])

print("Thinned Unique Phrases (Candidate Keyphrases): \n")
print(unique_phrases)

Thinned Unique Phrases (Candidate Keyphrases): 

[['mark'], ['information', 'system'], ['software', 'engineering', 'objective'], ['definition'], ['software', 'development', 'life', 'cycle'], ['software', 'process', 'model', 'waterfall', 'model'], ['spiral', 'model'], ['agile', 'model'], ['software', 'requirement'], ['sr'], ['feasibility', 'analysis'], ['software', 'design'], ['context', 'diagram'], ['dfd'], ['physical'], ['logical', 'dfds'], ['data', 'dictionary'], ['er', 'diagram'], ['decision', 'tree'], ['decision', 'table'], ['structure', 'chart'], ['english'], ['software', 'testing'], ['level'], ['white-box'], ['black-box', 'testing'], ['test', 'case', 'generation'], ['acceptance', 'testing'], ['software', 'validation'], ['regression', 'testing'], ['mutation', 'analysis'], ['cyclomatic', 'complexity'], ['reliability', 'concept'], ['software', 'reliability'], ['hazard'], ['mttf'], ['mtbf'], ['repair'], ['availability'], ['software', 'quality'], ['quality', 'attribute'], ['risk', 'ma

In [23]:
phrase_scores = []
keywords = []
for phrase in unique_phrases:
    phrase_score=0
    keyword = ''
    for word in phrase:
        keyword += str(word)
        keyword += " "
        phrase_score+=score[vocabulary.index(word)]
    phrase_scores.append(phrase_score)
    keywords.append(keyword.strip())

i=0
for keyword in keywords:
    print ("Keyword: '"+str(keyword)+"', Score: "+str(phrase_scores[i]))
    i+=1


Keyword: 'mark', Score: 0.5299281477928162
Keyword: 'information system', Score: 1.6040415167808533
Keyword: 'software engineering objective', Score: 9.02166336774826
Keyword: 'definition', Score: 0.7342217564582825
Keyword: 'software development life cycle', Score: 9.721467792987823
Keyword: 'software process model waterfall model', Score: 12.718610227108002
Keyword: 'spiral model', Score: 2.593160331249237
Keyword: 'agile model', Score: 2.578522562980652
Keyword: 'software requirement', Score: 8.256574988365173
Keyword: 'sr', Score: 0.7377454042434692
Keyword: 'feasibility analysis', Score: 2.7169315218925476
Keyword: 'software design', Score: 8.283243000507355
Keyword: 'context diagram', Score: 2.3573814630508423
Keyword: 'dfd', Score: 0.8693903684616089
Keyword: 'physical', Score: 0.8997173309326172
Keyword: 'logical dfds', Score: 1.8481943607330322
Keyword: 'data dictionary', Score: 1.820992648601532
Keyword: 'er diagram', Score: 2.437292754650116
Keyword: 'decision tree', Score: 

In [36]:
# Read the content of the text file
import pandas as pd

with open('/content/drive/MyDrive/B.TechProject/stop_words.txt', 'r') as file:
    lines = file.readlines()

# Initialize lists to store data
topics = []
labels = []

# Process lines to extract topic and label information
current_topic = None
for line in lines:
    line = line.strip()
    if line.startswith('Topic'):
        current_topic = line
    elif line:
        # Extract label information
        label = line.split('. ')[-1]
        topics.append(current_topic)
        labels.append(label)
print(label)

# Create a Pandas DataFrame
df = pd.DataFrame({'topic': topics, 'label': labels})
print(df)

you've
    topic     label
0    None         a
1    None     about
2    None     above
3    None     after
4    None     again
..    ...       ...
164  None     would
165  None  wouldn't
166  None       you
167  None    you're
168  None    you've

[169 rows x 2 columns]


In [39]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC


random_bloomsLevel=[]
print(pd.read_csv('/content/drive/MyDrive/B.TechProject/data2.txt', sep=';', header=None, names=['Text', 'Label'] , encoding='unicode_escape'))
data = pd.read_csv('/content/drive/MyDrive/B.TechProject/data2.txt', sep=';', header=None, names=['Text', 'Label'] , encoding='unicode_escape')
# Check for missing values and handle them
# data.dropna(subset=['Text', 'Label'], inplace=True)  # Drop rows with missing values
# print(data)
from sklearn.impute import SimpleImputer

# Create an imputer object
imputer = SimpleImputer(strategy='most_frequent')

# Impute missing values in 'Text' and 'Label' columns
# data[['Text', 'Label']] = imputer.fit_transform(data[['Text', 'Label']])

X_train, X_test, y_train, y_test = train_test_split(data['Text'], data['Label'], test_size=0.1, random_state=42)
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)
classifier = SVC(kernel='linear', C=1, random_state=42)
classifier.fit(X_train, y_train)
accuracy = classifier.score(X_test, y_test)
print('Accuracy:', accuracy)
def classify_input(text):
    vectorized_text = vectorizer.transform([text])
    label = classifier.predict(vectorized_text)[0]
    return label
for keyword in keywords:
  level=classify_input(keyword)
  sentence=f'{level} {keyword}'
  random_bloomsLevel.append(sentence)
print(random_bloomsLevel)

                                                  Text      Label
0    About what proportion of the population of the...  Knowledge
1    Correctly label the brain lobes indicated on t...  Knowledge
2                            Define compound interest.  Knowledge
3                   Define four types of traceability.  Knowledge
4                                 Define mercantilism.  Knowledge
..                                                 ...        ...
370  Analyze the role of empathy in conflict resolu...   Analysis
371  Analyze the effectiveness of specific interven...   Analysis
372  Analyze how socioeconomic status influences ac...   Analysis
373  Analyze the factors contributing to the digita...   Analysis
374  Analyze how historical narratives shape cultur...   Analysis

[375 rows x 2 columns]
Accuracy: 0.9473684210526315
['Knowledge mark', 'Knowledge information system', 'Comprehension software engineering objective', 'Knowledge definition', 'Comprehension software developm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

keyword to sentence using bert


In [40]:

import requests
bert_sentences=[]
keywords
url = "https://bloom-bert-api-dmkyqqzsta-as.a.run.app/predict"
for keyword in keywords:

    data = {"text": keyword}
    response = requests.post(url, json=data)
    data=response.json()['blooms_level']
    sentence = f'{data} {keyword} .'
    bert_sentences.append(sentence)





In [41]:
print(keywords)

['mark', 'information system', 'software engineering objective', 'definition', 'software development life cycle', 'software process model waterfall model', 'spiral model', 'agile model', 'software requirement', 'sr', 'feasibility analysis', 'software design', 'context diagram', 'dfd', 'physical', 'logical dfds', 'data dictionary', 'er diagram', 'decision tree', 'decision table', 'structure chart', 'english', 'software testing', 'level', 'white-box', 'black-box testing', 'test case generation', 'acceptance testing', 'software validation', 'regression testing', 'mutation analysis', 'cyclomatic complexity', 'reliability concept', 'software reliability', 'hazard', 'mttf', 'mtbf', 'repair', 'availability', 'software quality', 'quality attribute', 'risk management', 'quality factor', 'software quality assurance', 'quality standard', 'total quality management', 'software project management', 'software project planning', 'project scheduling', 'software configuration management', 'cost estimati

In [ ]:
import requests

# List to hold Bloom's levels
chatgpt_BloomsLevel = []

# List of keywords
keywords = [
    'information system', 'software engineering objective', 'definition',
    'software development life cycle', 'software process model waterfall model',
    'spiral model', 'agile model', 'software requirement', 'sr',
    'feasibility analysis', 'software design', 'context diagram', 'dfd',
    'physical', 'logical dfds', 'data dictionary', 'er diagram',
    'decision tree', 'decision table', 'structure chart', 'english',
    'software testing', 'level', 'white-box', 'black-box testing',
    'test', 'generation', 'acceptance testing', 'software validation',
    'regression testing', 'mutation analysis', 'cyclomatic complexity',
    'reliability concept', 'software reliability', 'hazard', 'mttf',
    'mtbf', 'repair', 'availability', 'software quality',
    'quality attribute', 'risk management', 'quality factor',
    'software quality assurance', 'quality standard',
    'total quality management', 'software project management',
    'software project planning', 'project scheduling',
    'software configuration management', 'cost estimation-cocomo',
    'function', 'halstead metric', 'project management tools- wb',
    'gantt chart', 'pert', 'critical path method'
]

# OpenAI API details
myurl = "https://api.openai.com/v1/chat/completions"
api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

# Loop through keywords and request Bloom's level
for keyword in keywords:
    data = {
        "model": "babbage-002",  # Specify the model you want to use
        "messages": [
            {
                "role": "user",
                "content": f'What is the Bloom\'s level of the topic "{keyword}"? Say it in a single word proper verb without punctuation.'
            }
        ],
        "temperature": 0.5,
        "max_tokens": 1000
    }

    # Make the API request
    response = requests.post(myurl, headers=headers, json=data)

    # Check if the response was successful
    if response.status_code == 200:
        level = response.json()['choices'][0]['message']['content'].strip()
        sentence = f'{level} {keyword}'
        chatgpt_BloomsLevel.append(sentence)
    else:
        print(f"Error: {response.status_code} - {response.text}")

# Print the results
print(chatgpt_BloomsLevel)


In [ ]:
  level=response.json()['choices'][0]['text']
  sentence= f'{level} {keyword}'
  sentence=sentence.replace('\n\n', '')
  chatgpt_BloomsLevel.append(sentence)

In [46]:
keywords.insert(0,'TOPICS')
random_bloomsLevel.insert(0,'Blooms  Level by SVM')
bert_sentences.insert(0,"Blooms  Level by BERT MODEL")
chatgpt_BloomsLevel.insert(0,"Blooms  Level by CHATGPT")

In [47]:

arr1=keywords
arr2=random_bloomsLevel
arr3=bert_sentences
arr4=chatgpt_BloomsLevel
arr5=['Manual Classification', 'Analyze information system', 'Apply software engineering objective', 'Remember definition', 'Analyze software development life cycle', 'Apply software process model waterfall model', 'Apply spiral model', 'Apply agile model', 'Analyze software requirement', 'Analyze sr', 'Analyze feasibility analysis', 'Synthesis software design', 'Analyze context diagram', 'Analyze dfd', 'Synthesis physical', 'Analyze logical dfds', 'Remember data dictionary', 'Analyze er diagram', 'Synthesis decision tree', 'Synthesis decision table', 'Synthesis structure chart', 'Comprehension english', 'Synthesis software testing', 'Evaluate level', 'Analyze white-box', 'Apply black-box testing', 'Apply test', 'Analysis generation', 'Analysis acceptance testing', 'Analysis software validation', 'Apply regression testing', 'Analyze mutation analysis', 'Synthesis cyclomatic complexity', 'Remembee reliability concept', 'Analysis software reliability', 'Evaluate hazard', 'Calculate mttf', 'Calculate mtbf', 'Remember repair', 'Analyze availability', 'Evaluate software quality', 'Evaluate quality attribute', 'Analyze risk management', 'Evaluate quality factor', 'Evaluate software quality assurance', 'Evaluate quality standard', 'Evaluate total quality management', 'Synthesis software project management', 'Analyze software project planning', 'Apply project scheduling', 'Analyze software configuration management', 'Analyze cost estimation-cocomo', 'Evaluate function', 'Synthesis halstead metric', 'Apply project management tools- wb', 'Synthesis gantt chart', 'Apply pert', 'Apply critical path method']

max_len_arr1 = max([len(s) for s in arr1])
max_len_arr2 = max([len(s) for s in arr2])
max_len_arr3 = max([len(s) for s in arr3])
max_len_arr4 = max([len(s) for s in arr4])
max_len_arr5 = max([len(s) for s in arr5])

length = 10
char = '-'
line = char * (max_len_arr1+max_len_arr2+max_len_arr3+max_len_arr4+40+max_len_arr5)
line1= char*(max_len_arr1+max_len_arr2)
line2= char*(max_len_arr1+max_len_arr3)
line3= char*(max_len_arr1+max_len_arr4)







In [48]:
print("-------------------SVM Method--------------------\n\n")

for i in range(len(arr1)):
    print(f"|{arr1[i]:<{max_len_arr1}}\t | {arr2[i]:<{max_len_arr2}}\t ")
    print(line1)

-------------------SVM Method--------------------


|TOPICS                                	 | Blooms  Level by SVM                              	 
----------------------------------------------------------------------------------------
|information system                    	 | Knowledge mark                                    	 
----------------------------------------------------------------------------------------
|software engineering objective        	 | Knowledge information system                      	 
----------------------------------------------------------------------------------------
|definition                            	 | Comprehension software engineering objective      	 
----------------------------------------------------------------------------------------
|software development life cycle       	 | Knowledge definition                              	 
----------------------------------------------------------------------------------------
|software process model

# New Section

# New Section

In [49]:
print("-------------------Bert Model--------------------\n\n")

for i in range(len(arr1)):
    print(f"|{arr1[i]:<{max_len_arr1}}\t | {arr3[i]:<{max_len_arr3}}\t ")
    print(line2)

-------------------Bert Model--------------------


|TOPICS                                	 | Blooms  Level by BERT MODEL                        	 
-----------------------------------------------------------------------------------------
|information system                    	 | Remember mark .                                    	 
-----------------------------------------------------------------------------------------
|software engineering objective        	 | Remember information system .                      	 
-----------------------------------------------------------------------------------------
|definition                            	 | Apply software engineering objective .             	 
-----------------------------------------------------------------------------------------
|software development life cycle       	 | Remember definition .                              	 
-----------------------------------------------------------------------------------------
|software pro

In [50]:
print("-------------------Chatgpt--------------------\n\n")

for i in range(len(arr1)):
    print(f"|{arr1[i]:<{max_len_arr1}}\t | {arr4[i]:<{max_len_arr4}}\t")
    print(line3)

-------------------Chatgpt--------------------


|TOPICS                                	 | Blooms  Level by CHATGPT	
--------------------------------------------------------------


IndexError: list index out of range

In [60]:
print(line)

for i in range(len(arr1)):
    print(f"| {arr1[i]:<{max_len_arr1}}\t | {arr2[i]:<{max_len_arr2}}\t | {arr3[i]:<{max_len_arr3}}\t  | {arr5[i]:<{max_len_arr5}}  ")
    print(line)

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| TOPICS                                	 | Blooms  Level by SVM                              	 | Blooms  Level by BERT MODEL                        	  | Manual Classification                         
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| information system                    	 | Knowledge mark                                    	 | Remember mark .                                    	  | Analyze information system                    
------------------------------------------------------------------------------------------------------